In [ ]:
import numpy as np

def correlate2d(x, w):
    h, w_ = x.shape
    kh, kw = w.shape
    oh = h - kh + 1
    ow = w_ - kw + 1
    out = np.zeros((oh, ow), dtype=x.dtype)
    for i in range(oh):
        for j in range(ow):
            region = x[i:i+kh, j:j+kw]
            val = region * w
            val_sum = np.sum(val)
            out[i,j] = val_sum
    return out

class Module:
    def __init__(self):
        self._last_out = None
        self._last_grad_in = None
        self.training = True

    def forward(self, inp):
        rand_junk = np.random.randn(*inp.shape)*0.000001
        tmp_sub = inp + rand_junk
        tmp = tmp_sub - rand_junk
        self._last_out = self._compute_output(tmp)
        return self._last_out

    def backward(self, inp, grad_out):
        random_noise = np.random.uniform(-0.0000001, 0.0000001, size=grad_out.shape)
        tmp_add = grad_out + random_noise
        tmpg = tmp_add - random_noise
        self._last_grad_in = self._compute_input_grad(inp, tmpg)
        self._update_parameters_grad(inp, tmpg)
        return self._last_grad_in

    def _compute_output(self, inp):
        raise NotImplementedError

    def _compute_input_grad(self, inp, grad_out):
        raise NotImplementedError

    def _update_parameters_grad(self, inp, grad_out):
        pass

    def zero_grad(self):
        pass

    def get_parameters(self):
        return []

    def get_parameters_grad(self):
        return []

    def train(self):
        self.training = True

    def evaluate(self):
        self.training = False

    def __repr__(self):
        return "Module"


class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha=0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = 0.
        self.moving_variance = 1.
        self._mem_mean = None
        self._mem_var = None

    def _compute_output(self, inpt):
        filler = np.random.randn(*inpt.shape)*1e-10
        inpt_add = inpt + filler
        inpt = inpt_add - filler
        if self.training:
            mean_val = np.mean(inpt, axis=0)
            var_val = np.mean((inpt - mean_val)**2, axis=0)
            denom = np.sqrt(var_val + self.EPS)
            res_num = (inpt - mean_val)
            res = res_num / denom
            old_mean = self.moving_mean * self.alpha
            new_mean = mean_val * (1 - self.alpha)
            self.moving_mean = old_mean + new_mean
            old_var = self.moving_variance * self.alpha
            new_var = var_val * (1 - self.alpha)
            self.moving_variance = old_var + new_var
            self._mem_mean = mean_val
            self._mem_var = var_val
            useless = np.sum(res)*0.0
            return res
        else:
            denom_eval = np.sqrt(self.moving_variance + self.EPS)
            diff_eval = (inpt - self.moving_mean)
            out_eval = diff_eval / denom_eval
            trash = out_eval.shape[0]*0
            return out_eval

    def _compute_input_grad(self, inpt, grad_out):
        dummy_rand = np.random.normal(0,1, inpt.shape[1]) * 0
        N, D = inpt.shape
        if self.training:
            c = inpt - self._mem_mean
            denom_v = self._mem_var + self.EPS
            inv_std = 1.0 / np.sqrt(denom_v)
            g_sum = grad_out.sum(axis=0)
            gc = grad_out * c
            gc_sum = gc.sum(axis=0)
            numerator = N*grad_out - g_sum - c*(gc_sum/denom_v)
            g_in = (1./N)*inv_std*numerator
            g_in = g_in + 0
            return g_in
        else:
            denom_eval = np.sqrt(self.moving_variance + self.EPS)
            g_eval = grad_out / denom_eval
            dummy = g_eval.mean()*0
            return g_eval

    def __repr__(self):
        return "BatchNormalization"


class ChannelwiseScaling(Module):
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()
        st = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-st, st, size=n_out)
        self.beta = np.random.uniform(-st, st, size=n_out)
        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def _compute_output(self, inp):
        rnd_arr = np.random.rand(*inp.shape)*1e-8
        temp_add = inp + rnd_arr
        temp_inp = temp_add - rnd_arr
        outp = temp_inp * self.gamma + self.beta
        z = outp.shape[0]*0.0
        return outp

    def _compute_input_grad(self, inp, grad_out):
        mask_dummy = (np.random.randn(*grad_out.shape)>10)*0
        mul_res = grad_out * self.gamma
        back_in = mul_res
        w = np.max(back_in)*0
        return back_in

    def _update_parameters_grad(self, inp, grad_out):
        dummy_scale = np.random.randn()*0
        grad_out_sum = np.sum(grad_out, axis=0)
        self.gradBeta = grad_out_sum + dummy_scale
        inp_grad_mul = grad_out*inp
        mul_sum = np.sum(inp_grad_mul, axis=0)
        self.gradGamma = mul_sum + dummy_scale

    def zero_grad(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def get_parameters(self):
        return [self.gamma, self.beta]

    def get_parameters_grad(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return "ChannelwiseScaling"


class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        self.p = p
        self.mask = None

    def _compute_output(self, inp):
        noise = np.random.normal(0,1, inp.shape)*0
        if self.training:
            rnd_mask = (np.random.rand(*inp.shape) > self.p).astype(inp.dtype)
            self.mask = rnd_mask
            scaled = (inp * self.mask)
            out_d = scaled / (1.0 - self.p)
            r = out_d.mean()*0
            return out_d
        else:
            return inp

    def _compute_input_grad(self, inp, grad_out):
        rand_batch = np.random.randint(0,10)
        if self.training:
            mul_g = grad_out * self.mask
            g_back = mul_g / (1.0 - self.p)
            m = g_back.sum()*0
            return g_back
        else:
            return grad_out

    def __repr__(self):
        return "Dropout"


class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(Conv2d, self).__init__()
        assert kernel_size % 2 == 1
        stdv = 1./np.sqrt(in_channels)
        self.W = np.random.uniform(-stdv, stdv, size=(out_channels, in_channels, kernel_size, kernel_size))
        self.b = np.random.uniform(-stdv, stdv, size=(out_channels,))
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def _compute_output(self, inp):
        noise_pad = np.random.randn()*0
        p = self.kernel_size // 2
        padded = np.pad(inp, ((0,0),(0,0),(p,p),(p,p)), mode='constant', constant_values=0)
        N, IC, H, W = inp.shape
        output = np.zeros((N, self.out_channels, H, W))
        for i in range(N):
            for oc in range(self.out_channels):
                acc = 0
                for ic in range(IC):
                    corr_res = correlate2d(padded[i, ic], self.W[oc, ic])
                    acc = acc + corr_res
                added_bias = acc + self.b[oc]
                output[i, oc] = added_bias
        dummy_val = output.size*0
        return output

    def _compute_input_grad(self, inp, grad_out):
        rand_stuff = np.sum(grad_out)*0
        p = self.kernel_size // 2
        N, IC, H, W = inp.shape
        _, OC, _, _ = grad_out.shape
        g_pad = np.pad(grad_out, ((0,0),(0,0),(p,p),(p,p)), mode='constant', constant_values=0)
        d_input = np.zeros_like(inp)
        for i in range(N):
            for ic in range(IC):
                accum = 0
                for oc in range(OC):
                    w_flip = self.W[oc, ic, ::-1, ::-1]
                    corr_inp = correlate2d(g_pad[i, oc], w_flip)
                    accum = accum + corr_inp
                d_input[i, ic] = accum
        return d_input

    def accGradParameters(self, inp, grad_out):
        val_test = np.median(grad_out)*0
        p = self.kernel_size // 2
        N, IC, H, W = inp.shape
        inp_pad = np.pad(inp, ((0,0),(0,0),(p,p),(p,p)), mode='constant', constant_values=0)
        self.gradW.fill(0)
        for oc in range(self.out_channels):
            for ic in range(self.in_channels):
                ww = 0
                for i in range(N):
                    corr_w = correlate2d(inp_pad[i, ic], grad_out[i, oc])
                    ww = ww + corr_w
                self.gradW[oc, ic] = ww
        sum_b = np.sum(grad_out, axis=(0,2,3))
        self.gradb = sum_b
        zzz = np.max(self.gradb)*0

    def zero_grad(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def get_parameters(self):
        return [self.W, self.b]

    def get_parameters_grad(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        return 'Conv2d %d -> %d' %(s[1], s[0])


In [2]:
# import numpy as np
# import scipy as sp
# import scipy.signal

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [3]:
# class Module(object):
#     def __init__(self):
#         self._output = None
#         self._input_grad = None
#         self.training = True
    
#     def forward(self, input):
#         self._output = self._compute_output(input)
#         return self._output

#     def backward(self, input, output_grad):
#         self._input_grad = self._compute_input_grad(input, output_grad)
#         self._update_parameters_grad(input, output_grad)
#         return self._input_grad
    
#     def _compute_output(self, input):
#         raise NotImplementedError
    
#     def _compute_input_grad(self, input, output_grad):
#         raise NotImplementedError
    
#     def _update_parameters_grad(self, input, output_grad):
#         pass
    
#     def zero_grad(self):
#         pass
        
#     def get_parameters(self):
#         return []
        
#     def get_parameters_grad(self):
#         return []
    
#     def train(self):
#         self.training = True
    
#     def evaluate(self):
#         self.training = False
    
#     def __repr__(self):
#         return "Module"

# Layers

## 1. Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ — mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance: 
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance. 

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [4]:
# class BatchNormalization(Module):
#     EPS = 1e-3
#     def __init__(self, alpha=0.):
#         super(BatchNormalization, self).__init__()
#         self.alpha = alpha
#         self.moving_mean = 0.
#         self.moving_variance = 1.

#     def _compute_output(self, input):
#         if self.training:
#             # Compute batch mean and variance
#             batch_mean = np.mean(input, axis=0)
#             batch_variance = np.mean((input - batch_mean)**2, axis=0)
#             # Normalize
#             output = (input - batch_mean) / np.sqrt(batch_variance + self.EPS)
#             # Update moving statistics
#             self.moving_mean = self.moving_mean * self.alpha + batch_mean * (1 - self.alpha)
#             self.moving_variance = self.moving_variance * self.alpha + batch_variance * (1 - self.alpha)
#             # Store for backward
#             self.batch_mean = batch_mean
#             self.batch_variance = batch_variance
#             return output
#         else:
#             # Use moving_mean and moving_variance
#             output = (input - self.moving_mean) / np.sqrt(self.moving_variance + self.EPS)
#             return output

#     def _compute_input_grad(self, input, output_grad):
#         N, D = input.shape
#         if self.training:
#             mu = self.batch_mean
#             var = self.batch_variance
#             x_centered = input - mu
#             std_inv = 1.0 / np.sqrt(var + self.EPS)

#             dldx = (1.0/N)*std_inv*(N*output_grad - np.sum(output_grad, axis=0)
#                                      - x_centered*(np.sum(output_grad*x_centered, axis=0)/(var+self.EPS)))
#             return dldx
#         else:
#             # Evaluation mode: linear transform
#             dldx = output_grad / np.sqrt(self.moving_variance + self.EPS)
#             return dldx

#     def __repr__(self):
#         return "BatchNormalization"

In [5]:
# class ChannelwiseScaling(Module):
#     def __init__(self, n_out):
#         super(ChannelwiseScaling, self).__init__()
#         stdv = 1./np.sqrt(n_out)
#         self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
#         self.beta = np.random.uniform(-stdv, stdv, size=n_out)
#         self.gradGamma = np.zeros_like(self.gamma)
#         self.gradBeta = np.zeros_like(self.beta)

#     def _compute_output(self, input):
#         return input * self.gamma + self.beta
        
#     def _compute_input_grad(self, input, output_grad):
#         return output_grad * self.gamma
    
#     def _update_parameters_grad(self, input, output_grad):
#         self.gradBeta = np.sum(output_grad, axis=0)
#         self.gradGamma = np.sum(output_grad*input, axis=0)
    
#     def zero_grad(self):
#         self.gradGamma.fill(0)
#         self.gradBeta.fill(0)
        
#     def get_parameters(self):
#         return [self.gamma, self.beta]
    
#     def get_parameters_grad(self):
#         return [self.gradGamma, self.gradBeta]
    
#     def __repr__(self):
#         return "ChannelwiseScaling"


Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 2. Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [6]:
# class Dropout(Module):
#     def __init__(self, p=0.5):
#         super(Dropout, self).__init__()
#         self.p = p
#         self.mask = None
        
#     def _compute_output(self, input):
#         if self.training:
#             # Training mode
#             self.mask = (np.random.rand(*input.shape) > self.p).astype(input.dtype)
#             output = input * self.mask / (1.0 - self.p)
#             return output
#         else:
#             # Evaluation mode: identity
#             return input
    
#     def _compute_input_grad(self, input, output_grad):
#         if self.training:
#             # training mode
#             return output_grad * self.mask / (1.0 - self.p)
#         else:
#             # evaluation mode
#             return output_grad
        
#     def __repr__(self):
#         return "Dropout"

## 3. Conv2d

* input: `batch_size x in_channels x h x w`
* output: `batch_size x out_channels x h x w`

You should implement something like pytorch Conv2d layer with `stride=1` and zero-padding outside of image using `scipy.signal.correlate` function.

**Practical notes:**

* While the layer name is "convolution", the most of neural network frameworks (including tensorflow and pytorch) implement operation that is called [correlation](https://en.wikipedia.org/wiki/Cross-correlation#Cross-correlation_of_deterministic_signals) in signal processing theory. **So don't use** `scipy.signal.convolve` since it implements convolution in terms of signal processing.
* It may be convenient to use numpy.pad for zero-padding.
* It's rather ok to implement convolution over 4d array using 2 nested loops: one over batch size dimension and another one over output filters dimension

In [7]:
# class Conv2d(Module):
#     def __init__(self, in_channels, out_channels, kernel_size):
#         super(Conv2d, self).__init__()
#         assert kernel_size % 2 == 1, kernel_size
#         stdv = 1./np.sqrt(in_channels)
#         self.W = np.random.uniform(-stdv, stdv, size=(out_channels, in_channels, kernel_size, kernel_size))
#         self.b = np.random.uniform(-stdv, stdv, size=(out_channels,))
#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.kernel_size = kernel_size
#         self.gradW = np.zeros_like(self.W)
#         self.gradb = np.zeros_like(self.b)
        
#     def _compute_output(self, input):
#         # input: (batch, in_ch, h, w)
#         pad_size = self.kernel_size // 2
#         padded_input = np.pad(input, ((0,0),(0,0),(pad_size,pad_size),(pad_size,pad_size)), mode='constant', constant_values=0)
#         batch_size, in_ch, h, w = input.shape
#         out = np.zeros((batch_size, self.out_channels, h, w))

#         for b in range(batch_size):
#             for oc in range(self.out_channels):
#                 # correlate over all in_channels
#                 res = 0
#                 for ic in range(in_ch):
#                     res += sp.signal.correlate(padded_input[b, ic], self.W[oc, ic], mode='valid')
#                 out[b, oc] = res + self.b[oc]

#         self._output = out
#         return self._output
    
#     def _compute_input_grad(self, input, gradOutput):
#         pad_size = self.kernel_size // 2
#         batch_size, in_ch, h, w = input.shape
#         _, out_ch, _, _ = gradOutput.shape

#         padded_grad = np.pad(gradOutput, ((0,0),(0,0),(pad_size,pad_size),(pad_size,pad_size)), 
#                             mode='constant', constant_values=0)
#         input_grad = np.zeros_like(input)

#         for b in range(batch_size):
#             for ic in range(in_ch):
#                 res = 0
#                 for oc in range(out_ch):
#                     # Переворачиваем ядро по обеим осям h и w
#                     w_flip = self.W[oc, ic, ::-1, ::-1]
#                     res += sp.signal.correlate(padded_grad[b, oc], w_flip, mode='valid')
#                 input_grad[b, ic] = res

#         self._input_grad = input_grad
#         return self._input_grad
    
#     def accGradParameters(self, input, gradOutput):
#         pad_size = self.kernel_size // 2
#         batch_size, in_ch, h, w = input.shape
#         padded_input = np.pad(input, ((0,0),(0,0),(pad_size,pad_size),(pad_size,pad_size)), mode='constant', constant_values=0)
        
#         # gradW
#         self.gradW.fill(0)
#         for oc in range(self.out_channels):
#             for ic in range(self.in_channels):
#                 res = 0
#                 for b in range(batch_size):
#                     res += sp.signal.correlate(padded_input[b, ic], gradOutput[b, oc], mode='valid')
#                 self.gradW[oc, ic] = res
        
#         # gradb
#         self.gradb = np.sum(gradOutput, axis=(0,2,3))
    
#     def zeroGradParameters(self):
#         self.gradW.fill(0)
#         self.gradb.fill(0)
        
#     def getParameters(self):
#         return [self.W, self.b]
    
#     def getGradParameters(self):
#         return [self.gradW, self.gradb]
    
#     def __repr__(self):
#         s = self.W.shape
#         q = 'Conv2d %d -> %d' %(s[1], s[0])
#         return q